In [77]:
import pyodbc
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
class SQLConnection:
    """ SQL Connection Class

        Object that handles connection and querying of a database
    """

    def __init__(self, host, database):
        self.conn = pyodbc.connect(driver='{SQL Server}', server=host, database=database, trusted_connection='yes')

    def query_to_cursor(self, query_string):
        cursor = self.conn.cursor()
        cursor.execute(query_string)
        return cursor
    
    def query_to_df(self, query_string, index):
        df = pd.read_sql(query_string, conn.conn, index_col=index)
        return df

    def end(self):
        self.conn.close()

In [61]:
QUERY = 'SELECT * FROM gestures WHERE record BETWEEN %d AND %d;'

conn = SQLConnection(host='localhost', database='EMG_Gestures')

QUERY_SIZE = 'SELECT COUNT(record) FROM gestures;'

# dataframe = conn.query_to_df(QUERY_SIZE, '')

cursor_size = conn.query_to_cursor(QUERY_SIZE)
data_size = cursor_size.fetchval()

batch_number = 256

batch_size = data_size / batch_number

# a = 0
# b = 4096

# data = (a,b)

#data = conn.query_to_df(QUERY % data, 'record')


In [50]:
    a = 0
    b = 10
    
    data_range = (a,b)
    
    data = conn.query_to_df(QUERY % data_range, 'record')
    
    data = data.abs()
    
    # Check head rows
    print(data.head())
    
    # drop class from data for training
    train_X = data.drop(columns=['class'])
    
    #train_X = MinMaxScaler()
    rows = len(train_X)
    cols = len(train_X.columns)
    
    pre_scl = train_X.values.reshape(rows*cols, 1)
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(pre_scl)
    train_X_res = scaler.transform(train_X)
    
    train_X = train_X_res.reshape(rows, cols)
    
    train_X = pd.DataFrame(train_X)
    
    # Check head rows
    train_X.head()
    
    # convert to binarise 7 classes from 'class' column
    train_y = to_categorical(data['class'], num_classes = 8)
    
    # Check head rows after conversion
    print(train_y[0:5])
    
    print(train_X)

        channel1  channel2  channel3  channel4  channel5  channel6  channel7  \
record                                                                         
1        0.00083   0.00004   0.00031   0.00095   0.00061   0.00026   0.00013   
2        0.00001   0.00003   0.00003   0.00019   0.00007   0.00013   0.00002   
3        0.00001   0.00001   0.00001   0.00006   0.00001   0.00000   0.00001   
4        0.00002   0.00003   0.00004   0.00001   0.00005   0.00005   0.00005   
5        0.00011   0.00002   0.00004   0.00005   0.00013   0.00004   0.00002   

        channel8  class  
record                   
1        0.00002    2.0  
2        0.00000    5.0  
3        0.00000    1.0  
4        0.00005    4.0  
5        0.00010    3.0  
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]
          0         1         2         3         4         5         6  \
0  0.747368 -0.915789 -0.347368  1.000000  0.2

In [83]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping



#batch_size = 4096
#batch_number = 256
num_classes = 8
epochs = 20
training_split = 0.8


# Base Model
baseline_model = Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    Dense(16, activation = 'relu', input_shape=(8,)),
    Dense(16, activation = 'relu'),
    Dense(num_classes, activation='softmax')
])

baseline_model.compile(optimizer=RMSprop(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'categorical_crossentropy'])

# Base Model with Dropout
baseline_model_drop = Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    Dense(16, activation = 'relu', input_shape=(8,)),
    Dropout(0.3),
    Dense(16, activation = 'relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

baseline_model_drop.compile(optimizer=RMSprop(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'categorical_crossentropy'])


# Base Model Neuron Change
baseline_model_neuron = Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    Dense(32, activation = 'relu', input_shape=(8,)),
    Dense(32, activation = 'relu'),
    Dense(num_classes, activation='softmax')
])

baseline_model_neuron.compile(optimizer=RMSprop(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'categorical_crossentropy'])


# Base Model Layer Change
baseline_model_layers = Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    Dense(16, activation = 'relu', input_shape=(8,)),
    Dense(16, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(num_classes, activation='softmax')
])

baseline_model_layers.compile(optimizer=RMSprop(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'categorical_crossentropy'])



early_stopping_monitor = EarlyStopping(patience=3)


a = 0
b = batch_number*batch_size*training_split

data_range = (a,b)

data = conn.query_to_df(QUERY % data_range, 'record')

data = data.abs()

# Check head rows
#print(data.head())

# drop class from data for training
train_X = data.drop(columns=['class'])

#train_X = MinMaxScaler()
rows = len(train_X)
cols = len(train_X.columns)

pre_scl = train_X.values.reshape(rows*cols, 1)

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(pre_scl)
train_X_res = scaler.transform(train_X)

train_X = train_X_res.reshape(rows, cols)

train_X = pd.DataFrame(train_X)

# Check head rows
#train_X.head()

# convert to binarise 7 classes from 'class' column
train_y = to_categorical(data['class'], num_classes = 8)

# Check head rows after conversion
#train_y[0:5]


baseline_history = baseline_model.fit(train_X, train_y, epochs=20, validation_split=0.3, callbacks=[early_stopping_monitor])

dropout_history = baseline_model_drop.fit(train_X, train_y, epochs=20, validation_split=0.3, callbacks=[early_stopping_monitor])

neuron_history = baseline_model_neuron.fit(train_X, train_y, epochs=20, validation_split=0.3, callbacks=[early_stopping_monitor])

layer_history = baseline_model_layers.fit(train_X, train_y, epochs=20, validation_split=0.3, callbacks=[early_stopping_monitor])

Train on 587202 samples, validate on 251658 samples
Epoch 1/20
587202/587202 [==============================]587202/587202 [==============================] - 194s 331us/step - loss: 1.0979 - acc: 0.5848 - categorical_crossentropy: 1.0979 - val_loss: 1.0240 - val_acc: 0.6152 - val_categorical_crossentropy: 1.0240

Epoch 2/20
587202/587202 [==============================]587202/587202 [==============================] - 193s 328us/step - loss: 1.0116 - acc: 0.6139 - categorical_crossentropy: 1.0116 - val_loss: 1.0048 - val_acc: 0.6160 - val_categorical_crossentropy: 1.0048

Epoch 3/20
587202/587202 [==============================]587202/587202 [==============================] - 196s 334us/step - loss: 0.9935 - acc: 0.6203 - categorical_crossentropy: 0.9935 - val_loss: 0.9988 - val_acc: 0.6197 - val_categorical_crossentropy: 0.9988

Epoch 4/20
587202/587202 [==============================]587202/587202 [==============================] - 195s 333us/step - loss: 0.9853 - acc: 0.6237 - catego

587202/587202 [==============================]587202/587202 [==============================] - 95s 162us/step - loss: 1.0046 - acc: 0.6163 - categorical_crossentropy: 1.0046 - val_loss: 1.0649 - val_acc: 0.5829 - val_categorical_crossentropy: 1.0649

Epoch 3/20
587202/587202 [==============================]587202/587202 [==============================] - 97s 165us/step - loss: 0.9915 - acc: 0.6218 - categorical_crossentropy: 0.9915 - val_loss: 1.0107 - val_acc: 0.6206 - val_categorical_crossentropy: 1.0107

Epoch 4/20
587202/587202 [==============================]587202/587202 [==============================] - 96s 163us/step - loss: 0.9860 - acc: 0.6240 - categorical_crossentropy: 0.9860 - val_loss: 1.0244 - val_acc: 0.6145 - val_categorical_crossentropy: 1.0244

Epoch 5/20
587202/587202 [==============================]587202/587202 [==============================] - 95s 161us/step - loss: 0.9837 - acc: 0.6245 - categorical_crossentropy: 0.9837 - val_loss: 0.9787 - val_acc: 0.6253 - v

In [81]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping



#batch_size = 4096
#batch_number = 256
num_classes = 8
epochs = 20



# Base Model
baseline_model = Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    Dense(16, activation = 'relu', input_shape=(8,)),
    Dense(16, activation = 'relu'),
    Dense(num_classes, activation='softmax')
])

baseline_model.compile(optimizer=RMSprop(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'categorical_crossentropy'])

# Base Model with Dropout
baseline_model_drop = Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    Dense(16, activation = 'relu', input_shape=(8,)),
    Dropout(0.3),
    Dense(16, activation = 'relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

baseline_model_drop.compile(optimizer=RMSprop(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'categorical_crossentropy'])


# Base Model Neuron Change
baseline_model_neuron = Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    Dense(32, activation = 'relu', input_shape=(8,)),
    Dense(32, activation = 'relu'),
    Dense(num_classes, activation='softmax')
])

baseline_model_neuron.compile(optimizer=RMSprop(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'categorical_crossentropy'])


# Base Model Layer Change
baseline_model_layers = Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    Dense(16, activation = 'relu', input_shape=(8,)),
    Dense(16, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(num_classes, activation='softmax')
])

baseline_model_layers.compile(optimizer=RMSprop(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'categorical_crossentropy'])


# model = Sequential()

# #get number of columns in training data
# #n_cols = train_X.shape[1]

# model.add(Dense(10, activation='relu', input_shape=(8,)))
# model.add(Dropout(0.1))
# model.add(Dense(10, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(num_classes, activation='softmax'))

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

# model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy', 'categorical_crossentropy'])

# model.summary()

for i in range(round(batch_number*0.8)):
    print("Batch: ")
    print(i)
    a = i*batch_size 
    b = (i+1)*batch_size 
    
    data_range = (a,b)
    
    data = conn.query_to_df(QUERY % data_range, 'record')
    
    data = data.abs()
    
    # Check head rows
    #print(data.head())
    
    # drop class from data for training
    train_X = data.drop(columns=['class'])
    
    #train_X = MinMaxScaler()
    rows = len(train_X)
    cols = len(train_X.columns)
    
    pre_scl = train_X.values.reshape(rows*cols, 1)
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(pre_scl)
    train_X_res = scaler.transform(train_X)
    
    train_X = train_X_res.reshape(rows, cols)
    
    train_X = pd.DataFrame(train_X)
    
    # Check head rows
    train_X.head()
    
    # convert to binarise 7 classes from 'class' column
    train_y = to_categorical(data['class'], num_classes = 8)
    
    # Check head rows after conversion
    train_y[0:5]

    
    baseline_history = model.fit(train_X, train_y, epochs=20, validation_split=0.3, callbacks=[early_stopping_monitor])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_68 (Dense)             (None, 10)                90        
_________________________________________________________________
dropout_50 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_51 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_70 (Dense)             (None, 8)                 88        
Total params: 288
Trainable params: 288
Non-trainable params: 0
_________________________________________________________________
Batch: 
0
Train on 2866 samples, validate on 1229 samples
Epoch 1/20
2866/2866 [==============================]2866/2866 [======================


Epoch 9/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 317us/step - loss: 1.3975 - acc: 0.4541 - categorical_crossentropy: 1.3975 - val_loss: 1.2967 - val_acc: 0.5569 - val_categorical_crossentropy: 1.2967

Epoch 10/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 241us/step - loss: 1.3898 - acc: 0.4601 - categorical_crossentropy: 1.3898 - val_loss: 1.2907 - val_acc: 0.5626 - val_categorical_crossentropy: 1.2907

Epoch 11/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 286us/step - loss: 1.3847 - acc: 0.4562 - categorical_crossentropy: 1.3847 - val_loss: 1.2775 - val_acc: 0.5618 - val_categorical_crossentropy: 1.2775

Epoch 12/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 293us/step - loss: 1.3807 - acc: 0.4688 - categorical_crossentropy: 1.3807 - val_loss: 1.2661 - val_acc: 0.5610 - val_categorical_crosse


Batch: 
3
Train on 2867 samples, validate on 1230 samples
Epoch 1/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 283us/step - loss: 1.2862 - acc: 0.5120 - categorical_crossentropy: 1.2862 - val_loss: 1.1796 - val_acc: 0.5642 - val_categorical_crossentropy: 1.1796

Epoch 2/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 307us/step - loss: 1.2775 - acc: 0.5044 - categorical_crossentropy: 1.2775 - val_loss: 1.1701 - val_acc: 0.5683 - val_categorical_crossentropy: 1.1701

Epoch 3/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 286us/step - loss: 1.2715 - acc: 0.5194 - categorical_crossentropy: 1.2715 - val_loss: 1.1764 - val_acc: 0.5545 - val_categorical_crossentropy: 1.1764

Epoch 4/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 301us/step - loss: 1.2780 - acc: 0.5253 - categorical_crossentropy: 1.2780 - val_

2867/2867 [==============================]2867/2867 [==============================] - 1s 335us/step - loss: 1.2122 - acc: 0.5441 - categorical_crossentropy: 1.2122 - val_loss: 1.1345 - val_acc: 0.5813 - val_categorical_crossentropy: 1.1345

Batch: 
7
Train on 2867 samples, validate on 1230 samples
Epoch 1/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 267us/step - loss: 1.2466 - acc: 0.5337 - categorical_crossentropy: 1.2466 - val_loss: 1.1028 - val_acc: 0.6081 - val_categorical_crossentropy: 1.1028

Epoch 2/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 272us/step - loss: 1.2364 - acc: 0.5337 - categorical_crossentropy: 1.2364 - val_loss: 1.0974 - val_acc: 0.6016 - val_categorical_crossentropy: 1.0974

Epoch 3/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 303us/step - loss: 1.2266 - acc: 0.5511 - categorical_crossentropy: 1.2266 - val_loss: 1.0981

2867/2867 [==============================]2867/2867 [==============================] - 1s 321us/step - loss: 1.1925 - acc: 0.5699 - categorical_crossentropy: 1.1925 - val_loss: 1.0693 - val_acc: 0.6114 - val_categorical_crossentropy: 1.0693

Epoch 5/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 282us/step - loss: 1.1867 - acc: 0.5657 - categorical_crossentropy: 1.1867 - val_loss: 1.0737 - val_acc: 0.6098 - val_categorical_crossentropy: 1.0737

Epoch 6/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 321us/step - loss: 1.1826 - acc: 0.5577 - categorical_crossentropy: 1.1826 - val_loss: 1.0689 - val_acc: 0.6073 - val_categorical_crossentropy: 1.0689

Batch: 
12
Train on 2867 samples, validate on 1230 samples
Epoch 1/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 380us/step - loss: 1.2316 - acc: 0.5319 - categorical_crossentropy: 1.2316 - val_loss: 1.107

...

2867/2867 [==============================]2867/2867 [==============================] - 1s 197us/step - loss: 1.1143 - acc: 0.5654 - categorical_crossentropy: 1.1143 - val_loss: 1.0231 - val_acc: 0.6106 - val_categorical_crossentropy: 1.0231

Epoch 8/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 256us/step - loss: 1.1156 - acc: 0.5734 - categorical_crossentropy: 1.1156 - val_loss: 1.0238 - val_acc: 0.6252 - val_categorical_crossentropy: 1.0238

Epoch 9/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 262us/step - loss: 1.1133 - acc: 0.5717 - categorical_crossentropy: 1.1133 - val_loss: 1.0263 - val_acc: 0.6106 - val_categorical_crossentropy: 1.0263

Epoch 10/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 359us/step - loss: 1.1125 - acc: 0.5741 - categorical_crossentropy: 1.1125 - val_loss: 1.0376 - val_acc: 0.6211 - val_categorical_crossentropy: 1.0376

2867/2867 [==============================]2867/2867 [==============================] - 1s 265us/step - loss: 1.1166 - acc: 0.5675 - categorical_crossentropy: 1.1166 - val_loss: 0.9917 - val_acc: 0.6236 - val_categorical_crossentropy: 0.9917

Epoch 2/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 321us/step - loss: 1.1038 - acc: 0.5800 - categorical_crossentropy: 1.1038 - val_loss: 0.9972 - val_acc: 0.6317 - val_categorical_crossentropy: 0.9972

Epoch 3/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 279us/step - loss: 1.0986 - acc: 0.5821 - categorical_crossentropy: 1.0986 - val_loss: 0.9907 - val_acc: 0.6333 - val_categorical_crossentropy: 0.9907

Epoch 4/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 356us/step - loss: 1.0914 - acc: 0.5860 - categorical_crossentropy: 1.0914 - val_loss: 0.9906 - val_acc: 0.6276 - val_categorical_crossentropy: 0.9906


2867/2867 [==============================]2867/2867 [==============================] - 1s 328us/step - loss: 1.1016 - acc: 0.5685 - categorical_crossentropy: 1.1016 - val_loss: 1.0333 - val_acc: 0.6179 - val_categorical_crossentropy: 1.0333

Epoch 4/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 356us/step - loss: 1.0932 - acc: 0.5762 - categorical_crossentropy: 1.0932 - val_loss: 1.0420 - val_acc: 0.6211 - val_categorical_crossentropy: 1.0420

Batch: 
193
Train on 2867 samples, validate on 1230 samples
Epoch 1/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 286us/step - loss: 1.1421 - acc: 0.5605 - categorical_crossentropy: 1.1421 - val_loss: 1.0640 - val_acc: 0.5911 - val_categorical_crossentropy: 1.0640

Epoch 2/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 310us/step - loss: 1.1107 - acc: 0.5710 - categorical_crossentropy: 1.1107 - val_loss: 1.05

2867/2867 [==============================]2867/2867 [==============================] - 1s 345us/step - loss: 1.1329 - acc: 0.5692 - categorical_crossentropy: 1.1329 - val_loss: 1.0829 - val_acc: 0.6073 - val_categorical_crossentropy: 1.0829

Epoch 6/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 338us/step - loss: 1.1365 - acc: 0.5678 - categorical_crossentropy: 1.1365 - val_loss: 1.0855 - val_acc: 0.6089 - val_categorical_crossentropy: 1.0855

Batch: 
199
Train on 2867 samples, validate on 1230 samples
Epoch 1/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 302us/step - loss: 1.0776 - acc: 0.5996 - categorical_crossentropy: 1.0776 - val_loss: 0.9959 - val_acc: 0.6317 - val_categorical_crossentropy: 0.9959

Epoch 2/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 280us/step - loss: 1.0676 - acc: 0.6034 - categorical_crossentropy: 1.0676 - val_loss: 0.99


Epoch 5/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 284us/step - loss: 1.0885 - acc: 0.5787 - categorical_crossentropy: 1.0885 - val_loss: 1.0586 - val_acc: 0.6033 - val_categorical_crossentropy: 1.0586

Epoch 6/20
2867/2867 [==============================]2867/2867 [==============================] - 1s 322us/step - loss: 1.0885 - acc: 0.5835 - categorical_crossentropy: 1.0885 - val_loss: 1.0541 - val_acc: 0.6008 - val_categorical_crossentropy: 1.0541



In [84]:
def plot_history(histories, key='binary_crossentropy'):
  plt.figure(figsize=(16,10))
    
  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])


plot_history([('baseline', baseline_history),
              ('dropout', dropout_history),
              ('neuron', neuron_history),
              ('layers', layers_history)])

NameError: name 'layers_history' is not defined

In [ ]:
conn.close()

In [20]:
df = pd.DataFrame(np.arange(12).reshape(3,4), columns=['A', 'B', 'C', 'D'])
print(df)
df = df.drop(columns=['B', 'C'])
print(df)

   A  B   C   D
0  0  1   2   3
1  4  5   6   7
2  8  9  10  11
   A   D
0  0   3
1  4   7
2  8  11
